In [34]:
import numpy as np
from sklearn.datasets import load_iris,load_boston
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature  # feature used for the split
        self.threshold = threshold  # threshold used for the split
        self.left = left  # left sub-tree
        self.right = right  # right sub-tree
        self.value = value  # predicted value of the leaf

Decision Tree for Classification case

In [6]:
class DecisionTreeClassifierFromScratch:

    def __init__(self, max_depth=None,min_samples_split=None):
        self.max_depth = max_depth  # maximum depth of the shaft
        self.min_samples_split = min_samples_split  # minimum samples required to split a node
        self.root = None  # tree root

    def fit(self, X, y):
        self.root = self._build_tree(X, y)

    def predict(self, X):
        return [self._predict(x, self.root) for x in X]
    
    def _build_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_classes = len(set(y))

        # Stop building the tree if the maximum depth is reached
        if self.max_depth is not None and depth >= self.max_depth:
            value = self._most_common_label(y)
            return Node(value=value)

        # Stop building the tree if all classes are identical
        if len(set(y)) == 1:
            return Node(value=y[0])
        
        # Stop building the tree if the number of samples in the node is less than min_samples_split
        if self.min_samples_split is not None:
            if n_samples < self.min_samples_split:
                value = self._most_common_label(y)
                return Node(value=value)

        # Find the best feature and threshold for the split
        best_feature, best_threshold = self._best_split(X, y, n_samples, n_features, n_classes)

        # Divide the data according to the best feature and the best threshold
        left_indices, right_indices = self._split(X[:, best_feature], best_threshold)

        # Build the left and right sub-tree
        left = self._build_tree(X[left_indices, :], y[left_indices], depth+1)
        right = self._build_tree(X[right_indices, :], y[right_indices], depth+1)

        # Return the decision node
        return Node(feature=best_feature, threshold=best_threshold, left=left, right=right)

    def _gini_impurity(self, y, n_classes):
        # Gini impurity computation
        return 1.0 - sum((np.sum(y == c) / len(y)) ** 2 for c in range(n_classes))

    def _best_split(self, X, y, n_samples, n_features, n_classes):
        best_gini = float('inf')
        best_feature = None
        best_threshold = None

        # Go through each feature to find the best split
        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            # Search for possible thresholds for each feature
            for threshold in thresholds:
                left_indices, right_indices = self._split(X[:, feature], threshold)
                # Ignore splits that have an empty side
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                # Impurity computation
                gini = (len(left_indices) / n_samples) * self._gini_impurity(y[left_indices],n_classes) + \
                     (len(right_indices) / n_samples) * self._gini_impurity(y[right_indices],n_classes)
                # Update the result if the impurity is better
                if gini < best_gini:
                    best_gini = gini
                    best_feature = feature
                    best_threshold = threshold
        # Return the best feature and its threshold
        return best_feature, best_threshold

    def _split(self, feature, threshold):
        # Definition of the indices of the elements included in the left or right split
        left_indices = np.where(feature <= threshold)[0]
        right_indices = np.where(feature > threshold)[0]
        return left_indices, right_indices

    def _most_common_label(self, y):
        # Find the main class of the region
        return sorted([(c, np.sum(y == c)) for c in set(y)], key=lambda x: x[1])[-1][0]

    def _predict(self, x, node):
        # If the node is a 'leaf', return the value of the node
        if node.value is not None:
            return node.value
        # If the sample feature value is less than or equal to the threshold, continue in the left sub-tree
        if x[node.feature] <= node.threshold:
            return self._predict(x, node.left)
        # Otherwise, continue in the right sub-tree
        else:
            return self._predict(x, node.right)

In [7]:
# In the case of classification we chose to test our decision tree with the IRIS dataset

# Loading the data
iris = load_iris()
# Splitting the data
X_train, X_test, y_train, y_test = model_selection.train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)
print('len of X_train: ',len(X_train),' ; len of X_test: ',len(X_test))

len of X_train:  120  ; len of X_test:  30


In [29]:
# From scratch 

# Fitting the decision tree
tree = DecisionTreeClassifierFromScratch(max_depth=4,min_samples_split=2)
tree.fit(X_train, y_train)

# Make predictions
predictions = tree.predict(X_test)
print(predictions)
print(list(y_test))

# Checking accuracy
accuracy = np.mean(predictions == y_test)

print(f"Accuracy: {accuracy}")
# One mistake on the 5th element

[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]
[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]
Accuracy: 1.0


In [30]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# With sklearn package

clf = DecisionTreeClassifier(random_state=42,max_depth=4)
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=4, random_state=42)

In [31]:
y_pred = clf.predict(X_test)
print(y_pred)
print(y_test)
accuracy = accuracy_score(y_test, y_pred)

print('Accuracy:', accuracy)

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Accuracy: 1.0


Decision tree for Regression case

In [32]:
class DecisionTreeRegressorFromScratch:

    def __init__(self, max_depth=None,min_samples_split=None):
        self.max_depth = max_depth  # maximum depth of the shaft
        self.min_samples_split = min_samples_split  # minimum samples required to split a node
        self.root = None  # tree root

    def fit(self, X, y):
        self.root = self._build_tree(X, y)

    def predict(self, X):
        return [self._predict(x, self.root) for x in X]
    
    def _build_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape

        # Stop building the tree if the maximum depth is reached
        if self.max_depth is not None and depth >= self.max_depth:
            value = np.mean(y)
            return Node(value=value)

        # Stop building the tree if all values are identical
        if np.all(y == y[0]):
            return Node(value=y[0])

        # Stop building the tree if the number of samples in the node is less than min_samples_split
        if self.min_samples_split is not None:
            if n_samples < self.min_samples_split:
                value = self._most_common_label(y)
                return Node(value=value)
            
        # Find the best feature and threshold for the split
        best_feature, best_threshold = self._best_split(X, y, n_samples, n_features)

        # Divide the data according to the best feature and the best threshold
        left_indices, right_indices = self._split(X[:, best_feature], best_threshold)

        # Build the left and right sub-tree
        left = self._build_tree(X[left_indices, :], y[left_indices], depth+1)
        right = self._build_tree(X[right_indices, :], y[right_indices], depth+1)

        # Return the decision node, with the best feature and threshold
        return Node(feature=best_feature, threshold=best_threshold, left=left, right=right)

    def _mse(self, y):
        # Computing MSE
        return np.mean((y - np.mean(y)) ** 2)

    def _best_split(self, X, y, n_samples, n_features):
        best_mse = float('inf')
        best_feature = None
        best_threshold = None

        # Go through each feature to find the best split
        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            # Search for possible thresholds for each feature
            for threshold in thresholds:
                left_indices, right_indices = self._split(X[:, feature], threshold)
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                # MSE computation
                mse = (len(left_indices) / n_samples) * self._mse(y[left_indices]) + \
                      (len(right_indices) / n_samples) * self._mse(y[right_indices])
                # Update the results if the MSE is better
                if mse < best_mse:
                    best_mse = mse
                    best_feature = feature
                    best_threshold = threshold
        # Return the best feature and its threshold
        return best_feature, best_threshold

    def _split(self, feature, threshold):
        # Definition of the indices of the elements included in the left or right split
        left_indices = np.where(feature <= threshold)[0]
        right_indices = np.where(feature > threshold)[0]
        return left_indices, right_indices
    
    def _predict(self, x, node):
        # If the node is a 'leaf', return the value of the node
        if node.value is not None:
            return node.value
        # If the sample feature value is less than or equal to the threshold, continue in the left sub-tree
        if x[node.feature] <= node.threshold:
            return self._predict(x, node.left)
        # Otherwise, continue in the right sub-tree
        else:
            return self._predict(x, node.right)

In [33]:
# In the case of regression we took the Boston Housing dataset to test our decsion tree regressor

# Loading the dataset
X, y = load_boston(return_X_y=True)

# Splitting the data
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

/Users/amaury/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California hous

In [35]:
# From scratch

# Fitting the decision tree
tree = DecisionTreeRegressorFromScratch(max_depth=5)
tree.fit(X_train, y_train)

# Make predicitons
y_pred = tree.predict(X_test)
print(y_pred)

# Computing the MSE (as an accuracy indicator)
mse = mean_squared_error(y_test, y_pred)
rsquared = r2_score(y_test,y_pred)
print("Mean Squared Error: ", mse)
print("R-squared: ", rsquared)

[22.726190476190474, 30.05, 19.733333333333334, 20.355555555555554, 15.910526315789474, 22.726190476190474, 15.910526315789474, 15.910526315789474, 22.726190476190474, 20.355555555555554, 19.733333333333334, 19.733333333333334, 9.203703703703704, 22.726190476190474, 20.355555555555554, 24.45, 19.733333333333334, 9.203703703703704, 44.65555555555556, 14.135, 22.726190476190474, 22.726190476190474, 15.910526315789474, 25.985185185185188, 14.135, 14.135, 20.355555555555554, 14.135, 15.910526315789474, 20.355555555555554, 19.733333333333334, 22.726190476190474, 10.4, 20.355555555555554, 15.910526315789474, 15.910526315789474, 24.45, 20.355555555555554, 22.299999999999997, 22.726190476190474, 19.6, 25.985185185185188, 44.65555555555556, 20.355555555555554, 22.726190476190474, 14.135, 15.910526315789474, 22.726190476190474, 15.910526315789474, 30.05, 20.355555555555554, 33.1, 15.910526315789474, 25.985185185185188, 44.65555555555556, 22.726190476190474, 15.910526315789474, 30.05, 22.72619047

In [36]:
from sklearn.tree import DecisionTreeRegressor

# With sklearn package

dt = DecisionTreeRegressor(max_depth=5, random_state=42)
dt.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = dt.predict(X_test)
print(y_pred)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred)
rsquared = r2_score(y_test,y_pred)
print("Mean Squared Error:", mse)
print("R-squared: ", rsquared)

[22.72619048 30.05       19.73333333 20.35555556 15.91052632 22.72619048
 15.91052632 15.91052632 22.72619048 20.35555556 19.73333333 19.73333333
  9.2037037  22.72619048 20.35555556 24.45       19.73333333  9.2037037
 44.65555556 14.135      22.72619048 22.72619048 15.91052632 25.98518519
 14.135      14.135      20.35555556 14.135      15.91052632 20.35555556
 19.73333333 22.72619048 10.4        20.35555556 15.91052632 15.91052632
 33.1        20.35555556 22.3        22.72619048 19.6        25.98518519
 44.65555556 20.35555556 22.72619048 14.135      15.91052632 22.72619048
 15.91052632 30.05       20.35555556 33.1        15.91052632 25.98518519
 44.65555556 22.72619048 15.91052632 30.05       22.72619048 22.3
 25.98518519 33.1        30.05       20.35555556 30.05       15.91052632
 14.135      22.72619048 30.05       15.91052632 20.35555556 25.98518519
  9.2037037  20.35555556 22.72619048  9.2037037  22.72619048 44.65555556
 13.17       15.91052632 20.35555556 13.17       20.3555555